In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28))
        label = self.data.iloc[idx, 0]
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = CustomDataset(csv_file='mnist_train.csv', transform=transform)
test_dataset = CustomDataset(csv_file='mnist_test.csv', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

model = CNN().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
criterion = nn.NLLLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)



# Train the CNN model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")



train_model(model, train_loader, criterion, optimizer)

# Simulate power analysis attack
def power_analysis_attack(model, input_data):
    # Simulate power consumption for each layer of the CNN
    power_consumption = []
    for layer in model.children():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            # Randomly generate power consumption values for each parameter
            power = np.random.normal(loc=0.5, scale=0.1, size=layer.weight.data.shape)
            power_consumption.append(power)

    
    return power_consumption

# Generate sample input data (e.g., images)
sample_input = torch.randn(1, 1, 28, 28)

# Perform the power analysis attack
inferred_information = power_analysis_attack(model, sample_input)

# Print the inferred information (for demonstration purposes)
for i, power_data in enumerate(inferred_information):
    print(f"Inferred information from layer {i}: Mean Power: {np.mean(power_data)}, Std Deviation: {np.std(power_data)}")


Epoch 1/10, Loss: 0.20090712678656292
Epoch 2/10, Loss: 0.07879864260748978
Epoch 3/10, Loss: 0.06134995499076164
Epoch 4/10, Loss: 0.05598692358716303
Epoch 5/10, Loss: 0.055016495637580244
Epoch 6/10, Loss: 0.05527420518827254
Epoch 7/10, Loss: 0.05643012489873837
Epoch 8/10, Loss: 0.05925008647584064
Epoch 9/10, Loss: 0.06514525016693462
Epoch 10/10, Loss: 0.06325772559335406
Inferred information from layer 0: Mean Power: 0.5073970057813072, Std Deviation: 0.10290807425045675
Inferred information from layer 1: Mean Power: 0.5005830273279099, Std Deviation: 0.09914501736893884
Inferred information from layer 2: Mean Power: 0.4999689701651271, Std Deviation: 0.10005980860627747
Inferred information from layer 3: Mean Power: 0.49838948977448905, Std Deviation: 0.10030352119243989


In [7]:
class CNN1(nn.Module):
  def __init__(self):
    super(CNN1, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.dropout(x)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

    model = CNN1().to(device)
    optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
    criterion = nn.NLLLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs + torch.randn_like(inputs) * 0.1)  # Add Gaussian noise to input data
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

train_model(model, train_loader, criterion, optimizer)

# Simulate power analysis attack
def power_analysis_attack(model, input_data):
    # Simulate power consumption for each layer of the CNN
    power_consumption = []
    for layer in model.children():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            # Randomly generate power consumption values for each parameter
            power = np.random.normal(loc=0.5, scale=0.1, size=layer.weight.data.shape)
            power_consumption.append(power)

    
    # Return the inferred information (for demonstration purposes)
    return power_consumption

# Generate sample input data (e.g., images)
sample_input = torch.randn(1, 1, 28, 28)

# Perform the power analysis attack
inferred_information = power_analysis_attack(model, sample_input)

# Print the inferred information (for demonstration purposes)
for i, power_data in enumerate(inferred_information):
    print(f"Inferred information from layer {i}: Mean Power: {np.mean(power_data)}, Std Deviation: {np.std(power_data)}")

Epoch 1/10, Loss: 0.07124957521833708
Epoch 2/10, Loss: 0.07016634022824383
Epoch 3/10, Loss: 0.06970453110843541
Epoch 4/10, Loss: 0.07570873571849343
Epoch 5/10, Loss: 0.07434013260985321
Epoch 6/10, Loss: 0.07182656778460869
Epoch 7/10, Loss: 0.07529867452339041
Epoch 8/10, Loss: 0.07869881873921687
Epoch 9/10, Loss: 0.07824821860199647
Epoch 10/10, Loss: 0.0772801198110072
Inferred information from layer 0: Mean Power: 0.5031222196838636, Std Deviation: 0.09647020215223648
Inferred information from layer 1: Mean Power: 0.5009904816646004, Std Deviation: 0.09965654946890905
Inferred information from layer 2: Mean Power: 0.5000746500404761, Std Deviation: 0.09999111440240037
Inferred information from layer 3: Mean Power: 0.49480184502844243, Std Deviation: 0.10332559900396238
